In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import joblib                        # to save trained models
import os
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
import matplotlib

/mnt/nrdstor/lxu/mzhang23/mzconda_env/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [2]:
csv_dir = "/work/lxu/mzhang23/FlightSize_data_collection/csvs"
csvs = os.listdir(csv_dir)

In [3]:
dataframes = []
cnt = 0
for csv in csvs:
    file_path = os.path.join(csv_dir,csv)
    if csv.endswith(".csv"):
        df = pd.read_csv(file_path)
        df['flow_id']=cnt
        cnt+=1
        # df['flow_id']="".join([ele for ele in csv if ele.isdigit()])
        dataframes.append(df)
merged_dataframe = pd.concat(dataframes, ignore_index=True)
data = merged_dataframe.copy()
fullcolumns = data.columns.tolist()
data = data[data["skb.data_len"]>400]
# data.to_csv('data.csv', index=False)

In [4]:
data

,Time,FlightSizeRef,FlightSizeLin,Sequence,skb.data_len,icsk_ca_status,tp_snd_cwnd,tp_snd_ssthresh,tp_packets_out,tp_retrans_out,...,tp_rtt_seq,tp_srtt_us,tp_delivered,tp_lost,tp_total_retrans,tp_reordering,tp_rcv_wnd,tp_reord_seen,tp_segs_out,flow_id
0,1.005111,2,2,2153077,1448,0,3,2,2,0,...,2153077,389799,114,8,8,3,64256,0,126,0
1,1.035453,0,2,2154525,2896,0,4,2,2,0,...,2153077,386702,115,8,8,3,64256,0,128,0
2,1.046764,2,3,2157421,1448,0,4,2,3,0,...,2153077,388399,116,8,8,3,64256,0,129,0
3,1.059877,2,3,2158869,1448,0,4,2,3,0,...,2158869,389335,117,8,8,3,64256,0,130,0
4,1.079964,1,3,2160317,1448,0,4,2,3,0,...,2158869,385178,118,8,8,3,64256,0,131,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547954,29.925333,123,133,3177423674,2896,0,137,114,133,0,...,3177380234,9130588,2142,4,4,3,64256,0,2282,114
547955,29.925342,123,135,3177426570,2896,0,137,114,135,0,...,3177380234,9130588,2142,4,4,3,64256,0,2284,114
547956,29.979769,121,135,3177429466,1448,0,137,114,135,0,...,3177380234,9265639,2144,4,4,3,64256,0,2285,114
547957,29.979830,121,133,3177430914,2896,0,138,114,133,0,...,3177380234,9363731,2147,4,4,3,64256,0,2287,114


In [5]:
# -------------------------------------------------
# 2.  Feature / target / groups  -------------------
DROP_COLS   = ["Time", "Sequence", "tp_rtt_seq", "tp_rcv_tstamp"]
FS_COLUMNS  = ["FlightSizeRef", "FlightSizeLin"]              # later dropped
TARGET_COL  = "FlightSizeRef"
GROUP_COL   = "flow_id"        # <-- ***adjust to your real column name***

In [6]:
# *X* still contains FS columns for a moment; we’ll drop them *after* splitting
X_full = data.drop(columns=DROP_COLS)
y_full = data[TARGET_COL].values
groups = data[GROUP_COL].values

In [7]:
# -------------------------------
# 3.  Two‑Way Split: 80% Train, 20% Test
#     (Ensure no overlap of flow_ids)
# -------------------------------
gss = GroupShuffleSplit(test_size=0.20, n_splits=1, random_state=123)
train_idx, test_idx = next(gss.split(X_full, y_full, groups=groups))

In [8]:
X_train_wFS = X_full.iloc[train_idx]
X_test_wFS  = X_full.iloc[test_idx]
y_train     = y_full[train_idx]
y_test      = y_full[test_idx]
groups_train = groups[train_idx]
groups_test  = groups[test_idx]

In [9]:
print(f"Training rows: {len(train_idx)},  Test rows: {len(test_idx)}")
print(f"Unique flows in train: {np.unique(groups_train).shape[0]},  Unique flows in test: {np.unique(groups_test).shape[0]}")
print("Overlap between train/test flows:", len(set(groups_train) & set(groups_test)))  # should be 0


Training rows: 455815,  Test rows: 91940
Unique flows in train: 92,  Unique flows in test: 23
Overlap between train/test flows: 0


In [10]:
# -------------------------------
# 4.  Drop Flight‑Size & Group Columns
#     so that models cannot “peek”
# -------------------------------
X_train = X_train_wFS.drop(columns=FS_COLUMNS + [GROUP_COL])
X_test  = X_test_wFS.drop(columns=FS_COLUMNS + [GROUP_COL])

In [11]:
feature_names = X_train.columns.tolist()

In [12]:
# Linux baseline
Linux_fs  = X_test_wFS['FlightSizeLin']
print("Linux ➜  RMSE %.1f   R² %.3f"
      % (root_mean_squared_error(y_test, Linux_fs),
         r2_score(y_test, Linux_fs)))

Linux ➜  RMSE 108.3   R² 0.556


In [13]:
# ------------------------------------------------------------------
# 4‑A.  Decision‑Tree  ----------------------------------------------
dt = DecisionTreeRegressor(max_depth=10, random_state=42)
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10, random_state=42)

In [14]:
dt_pred = dt.predict(X_test)
print("Decision Tree ➜  RMSE %.1f   R² %.3f"
      % (root_mean_squared_error(y_test, dt_pred),
         r2_score(y_test, dt_pred)))

Decision Tree ➜  RMSE 108.7   R² 0.553


In [15]:
# ------------------------------------------------------------------
# 4‑B.  Random Forest  ----------------------------------------------
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [16]:
rf_pred = rf.predict(X_test)
print("Random Forest ➜  RMSE %.1f   R² %.3f"
      % (root_mean_squared_error(y_test, rf_pred),
         r2_score(y_test, rf_pred)))

Random Forest ➜  RMSE 109.8   R² 0.544


In [17]:
# # ------------------------------------------------------------------
# # 4‑C.  XGBoost  -----------------------------------------------------
# xgb = XGBRegressor(
#         n_estimators=800, max_depth=8, learning_rate=0.01,
#         subsample=0.9, colsample_bytree=0.9,
#         objective="reg:squarederror", tree_method="hist",
#         eval_metric="rmse", n_jobs=-1, random_state=42)
# xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

In [18]:
# ------------------------------------------------------------------
# 4‑C.  XGBoost  -----------------------------------------------------
xgb = XGBRegressor(
        n_estimators=866, max_depth=8, learning_rate=0.07,
        subsample=0.92, colsample_bytree=0.63,
        objective="reg:squarederror", tree_method="hist",
        eval_metric="rmse", n_jobs=-1, random_state=42)
xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.63, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.07, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=866,
             n_jobs=-1, num_parallel_tree=None, ...)

In [19]:
xgb_pred = xgb.predict(X_test)
print("XGBoost       ➜  RMSE %.1f   R² %.3f"
      % (root_mean_squared_error(y_test, xgb_pred),
         r2_score(y_test, xgb_pred)))

XGBoost       ➜  RMSE 75.7   R² 0.783


In [20]:
models = {"Random Forest": rf, "XGBoost": xgb, "Decision‑Tree": dt}
# models = {"XGBoost": xgb}

In [21]:
# -------------------------------
# 6.  5‑Fold Group K‑Fold CV on TRAIN Portion
# -------------------------------
gkf = GroupKFold(n_splits=5)
cv_results = {name: {"rmse": [], "r2": []} for name in models}

In [22]:
# print("\n===== 5‑Fold Group CV on 80% Train Portion =====")
# for fold, (tr_idx, val_idx) in enumerate(gkf.split(X_train, y_train, groups=groups_train), start=1):
#     X_tr_fold = X_train.iloc[tr_idx]
#     y_tr_fold = y_train[tr_idx]
#     X_val_fold = X_train.iloc[val_idx]
#     y_val_fold = y_train[val_idx]

#     for name, model in models.items():
#         model.fit(X_tr_fold, y_tr_fold)
#         preds = model.predict(X_val_fold)
#         rmse_val = np.sqrt(mean_squared_error(y_val_fold, preds))
#         r2_val = r2_score(y_val_fold, preds)

#         cv_results[name]["rmse"].append(rmse_val)
#         cv_results[name]["r2"].append(r2_val)

#     print(f" Fold {fold} completed.")
# # Print CV results (mean ± std)
# for name in models:
#     rmse_arr = cv_results[name]["rmse"]
#     r2_arr = cv_results[name]["r2"]
#     print(
#         f"{name:<12} ➜  RMSE {np.mean(rmse_arr):.2f} ± {np.std(rmse_arr):.2f} | "
#         f"R² {np.mean(r2_arr):.3f} ± {np.std(r2_arr):.3f}"
#     )



===== 5‑Fold Group CV on 80% Train Portion =====
 Fold 1 completed.
 Fold 2 completed.
 Fold 3 completed.
 Fold 4 completed.
 Fold 5 completed.
Random Forest ➜  RMSE 135.39 ± 87.22 | R² 0.763 ± 0.228
XGBoost      ➜  RMSE 75.45 ± 19.39 | R² 0.923 ± 0.051
Decision‑Tree ➜  RMSE 134.86 ± 88.34 | R² 0.760 ± 0.243


In [23]:
# -------------------------------
# 7.  Retrain on ENTIRE Train Set and Evaluate on Held‑Out Test Set
# -------------------------------
print("\n===== Retrain on Full 80% Train and Evaluate on 20% Test =====")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_r2 = r2_score(y_test, y_pred_test)

    print(f"{name:<12} ➜  Test RMSE {test_rmse:.2f} | Test R² {test_r2:.3f}")


===== Retrain on Full 80% Train and Evaluate on 20% Test =====
Random Forest ➜  Test RMSE 110.18 | Test R² 0.541
XGBoost      ➜  Test RMSE 75.68 | Test R² 0.783
Decision‑Tree ➜  Test RMSE 108.71 | Test R² 0.553


In [24]:
matplotlib.rcParams['pdf.fonttype'] = 42

In [34]:
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
# XGBoost importances
booster = xgb.get_booster()
xgb_imp_dict = booster.get_score(importance_type='gain')
# Map to full feature list
xgb_imp = np.array([xgb_imp_dict.get(f, 0) for f in feature_names], dtype=float)
xgb_idx = np.argsort(xgb_imp)[::-1][:10]
plt.figure(figsize=(5,4))
plt.bar(range(10), xgb_imp[xgb_idx])
plt.xticks(range(10), [feature_names[i].replace('status','state') for i in xgb_idx], rotation=25, ha='right')
plt.title("XGBoost – Top 10 Feature Importances")
plt.ylabel("Gain")
plt.tight_layout()
# plt.show()
plt.savefig("xgb2.pdf", bbox_inches='tight', pad_inches=0)
plt.clf()

<Figure size 500x400 with 0 Axes>

In [35]:
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
# Random Forest importances
rf_imp = rf.feature_importances_
rf_idx = np.argsort(rf_imp)[::-1][:10]
plt.figure(figsize=(5,4))
plt.bar(range(10), rf_imp[rf_idx])
plt.xticks(range(10), [feature_names[i].replace('status','state') for i in rf_idx], rotation=25, ha='right')
plt.title("Random Forest – Top 10 Feature Importances")
plt.ylabel("Gini importance")
plt.tight_layout()
# plt.show()
plt.savefig("rf2.pdf", bbox_inches='tight', pad_inches=0)
plt.clf()

<Figure size 500x400 with 0 Axes>

In [36]:
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
# DT importances
dt_imp = dt.feature_importances_
dt_idx = np.argsort(dt_imp)[::-1][:10]
plt.figure(figsize=(5,4))
plt.bar(range(10), dt_imp[rf_idx])
plt.xticks(range(10), [feature_names[i] for i in dt_idx], rotation=45, ha='right')
plt.title("Decision Tree – Top 10 Feature Importances")
plt.tight_layout()
# plt.show()
plt.savefig("dt2.pdf")
plt.clf()

<Figure size 500x400 with 0 Axes>

In [32]:
xgb_imp_dict = booster.get_score(importance_type='gain')
xgb_imp = np.array([xgb_imp_dict.get(f, 0) for f in feature_names], dtype=float)
xgb_idx = np.argsort(xgb_imp)[::-1][:10]

# --- 1. Export Plotting Data ---
# Extract the y-values (bar heights) and x-tick labels into variables
plot_y_values = xgb_imp[xgb_idx]
plot_x_labels = [feature_names[i].replace('status', 'state') for i in xgb_idx]

# --- 2. Print Data in List Format for Reuse ---
print("--- Data for Reuse ---")
print("X-Labels (Features):")
print(plot_x_labels)
print("\nY-Values (Gain):")
print(plot_y_values.tolist()) # .tolist() converts numpy array to a standard list
print("----------------------\n")


# --- 3. Generate Plot Using Exported Data ---
plt.figure(figsize=(5, 4))

# Use the extracted variables to create the plot
plt.bar(range(len(plot_x_labels)), plot_y_values)
plt.xticks(range(len(plot_x_labels)), plot_x_labels, rotation=25, ha='right')

plt.title("XGBoost – Top 10 Feature Importances")
plt.ylabel("Gain")
plt.tight_layout()
# plt.show()
plt.savefig("xgb2.pdf", bbox_inches='tight', pad_inches=0)
plt.clf()

--- Data for Reuse ---
X-Labels (Features):
['icsk_ca_state', 'tp_packets_out', 'tp_snd_cwnd', 'tp_lost', 'tp_lost_out', 'tp_total_retrans', 'tp_srtt_us', 'tp_sacked_out', 'tp_snd_ssthresh', 'tp_segs_out']

Y-Values (Gain):
[13784443.0, 6784691.5, 4725497.5, 3618343.75, 2559845.75, 2162222.0, 1419125.875, 1334991.25, 1298492.25, 1170891.125]
----------------------



<Figure size 500x400 with 0 Axes>

In [33]:
# --- Data Preparation (Original) ---
rf_imp = rf.feature_importances_
rf_idx = np.argsort(rf_imp)[::-1][:10]

# --- 1. Export Plotting Data into Variables ---
# Extract the y-values (bar heights) and x-tick labels
plot_y_values = rf_imp[rf_idx]
plot_x_labels = [feature_names[i].replace('status', 'state') for i in rf_idx]

# --- 2. Print Data in List Format for Reuse ---
print("--- Data for Reuse ---")
print("X-Labels (Features):")
print(plot_x_labels)
print("\nY-Values (Gini importance):")
print(plot_y_values.tolist()) # .tolist() converts numpy array to a standard list
print("----------------------\n")

# --- 3. Generate Plot Using Exported Data ---
plt.figure(figsize=(5,4))

# Use the extracted variables to create the plot
plt.bar(range(len(plot_x_labels)), plot_y_values)
plt.xticks(range(len(plot_x_labels)), plot_x_labels, rotation=25, ha='right')

plt.title("Random Forest – Top 10 Feature Importances")
plt.ylabel("Gini importance")
plt.tight_layout()
# plt.show()
plt.savefig("rf2.pdf", bbox_inches='tight', pad_inches=0)
plt.clf()

--- Data for Reuse ---
X-Labels (Features):
['tp_snd_cwnd', 'tp_packets_out', 'tp_srtt_us', 'tp_total_retrans', 'tp_lost', 'tp_segs_out', 'tp_sacked_out', 'tp_rttvar_us', 'tp_delivered', 'tp_snd_ssthresh']

Y-Values (Gini importance):
[0.6527005209542086, 0.15625826402380924, 0.15266215916026715, 0.016850392285959578, 0.010919275609581131, 0.0024877375324688657, 0.0021285253250149364, 0.0016485014940327144, 0.0014710368935402223, 0.0009721073286537964]
----------------------



<Figure size 500x400 with 0 Axes>